In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import statsmodels.api as sm
from scipy.stats import chi2_contingency
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error as mse
import datetime
import time
from datetime import timezone

In [2]:
# read in dataset
data = pd.read_csv('Occupancy.csv')

data

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,2015-02-02 14:19:00,23.7000,26.2720,585.200000,749.200000,0.004764,1
1,2015-02-02 14:19:59,23.7180,26.2900,578.400000,760.400000,0.004773,1
2,2015-02-02 14:21:00,23.7300,26.2300,572.666667,769.666667,0.004765,1
3,2015-02-02 14:22:00,23.7225,26.1250,493.750000,774.750000,0.004744,1
4,2015-02-02 14:23:00,23.7540,26.2000,488.600000,779.000000,0.004767,1
...,...,...,...,...,...,...,...
20555,2015-02-18 09:15:00,20.8150,27.7175,429.750000,1505.250000,0.004213,1
20556,2015-02-18 09:16:00,20.8650,27.7450,423.500000,1514.500000,0.004230,1
20557,2015-02-18 09:16:59,20.8900,27.7450,423.500000,1521.500000,0.004237,1
20558,2015-02-18 09:17:59,20.8900,28.0225,418.750000,1632.000000,0.004279,1


In [3]:
# check for missing values
data.isna().sum()

date             0
Temperature      0
Humidity         0
Light            0
CO2              0
HumidityRatio    0
Occupancy        0
dtype: int64

In [4]:
# dependent variable (Y) is Occupancy (binary 0/1)
# independent variables (X) are continuous:
#     Temperature, Humidity, Light, CO2, HumidityRatio are float

# create dataframe to hold all results and display together at end
# ph, ph_index, mu, n_s, mse
allresults = pd.DataFrame({'ph': pd.Series(dtype='int'),
                   'ph_index': pd.Series(dtype='int'),
                   'mu': pd.Series(dtype='float'),
                    'n_s': pd.Series(dtype='int'),
                   'mse': pd.Series(dtype='float'),
                          })

df = data.copy()

# date happens every minute, so use index number for our time series as it represents 1 minute
df['date_index'] = np.arange(1, df.shape[0] + 1)

ts = pd.DataFrame(df.date_index)
ys = pd.DataFrame(df.Occupancy)


def timeseriesreg(ph,ph_index,mu,n_s):

    # Arrays to hold predicted values
    tp_pred = np.zeros(n_s-1) 
    yp_pred = np.zeros(n_s-1)
    
    # At every iteration of the for loop a new data sample is acquired
    for i in range(2, n_s+1):# start out with 2 leading datapoints
        #get x and y data "available" for our prediction
        ts_tmp = ts[0:i]
        ys_tmp = ys[0:i]
        ns = len(ys_tmp)
        
        weights = np.ones(ns)*mu
        for k in range(ns):
            #adjust weights to be downweighted according to their timestep away from our prediction
            weights[k] = weights[k]**k
        weights = np.flip(weights, 0)
            
        #perform linear regression on "available" data using the mu-adjusted weights
        lm_tmp = LinearRegression() 
        model_tmp = lm_tmp.fit(ts_tmp, ys_tmp, sample_weight=weights)
        # X (indep/predictor) is time, Y (dep/target) is occupancy
        
        #store model coefficients and intercepts to compute prediction
        m_tmp = model_tmp.coef_
        q_tmp = model_tmp.intercept_
    
        #use ph to make the model prediction according to the prediction time
        tp = ts.iloc[i-1,0] + ph
        yp = m_tmp*tp + q_tmp    
        tp_pred[i-2] = tp    
        # yp is a numpy array, must extract the element from the array to avoid a deprecation error
        yp_pred[i-2] = yp.item()

    # calculate mse
    mse0 = mse(ys['Occupancy'][int(ph_index):int(n_s+ph_index-1)],yp_pred)
    # print results
    #print(f"ph = {ph}, ph_index = {ph_index}, mu = {mu}, {n_s} samples: \n")
    #print(f"MSE is {mse0}")
    #print("\n\n")

    # print plot of results
    #fig, ax = plt.subplots(figsize=(10,10))
    #fig.suptitle('Occupancy Prediction', fontsize=22, fontweight='bold')
    #ax.set_title('mu = %g, ph=%g ' %(mu, ph))
    #ax.plot(tp_pred, yp_pred, label='Predicted Value') 
    #ax.plot(ts.iloc[0:n_s,0], ys.iloc[0:n_s,0], label='Occupancy data') 
    #ax.set_xlabel('Date Index')
    #ax.set_ylabel('Occupied (1) or Not (0)')
    #ax.legend()

    # save results to dataframe
    newrow = {"ph" : ph, "ph_index" : ph_index, "mu" : mu, "n_s" : n_s, "mse" : mse0}
    return newrow


def concat_results(allresults):
    # define all the values of ph, ph_index, mu, n_s to test
    phis = [10, 5, 1]
    phs = [60, 30, 10]
    mus = [0.1, 0.9, 1.0]
    nss = [500, 1000, 5000]
    # iterate through all the combinations
    for item1 in phs:
        ph = item1
        for item2 in mus:
            mu = item2
            for item3 in nss:
                n_s = item3
                for item4 in phis:
                    ph_index = item4
                    # run test
                    newrow = timeseriesreg(ph,ph_index,mu,n_s)
                    # concatenate test results
                    allresults = pd.concat([allresults, pd.DataFrame([newrow])], ignore_index=True)
    # sort by best mse and return
    allresults.sort_values(by=["mse"], inplace=True)
    return allresults


# run the function
allresults = concat_results(allresults)
# print the results
allresults


,ph,ph_index,mu,n_s,mse
68,10,1,0.9,1000,0.007944
71,10,1,0.9,5000,0.015416
65,10,1,0.9,500,0.015904
67,10,5,0.9,1000,0.022422
41,30,1,0.9,1000,0.030546
...,...,...,...,...,...
1,60,5,0.1,500,14.800458
0,60,10,0.1,500,15.066881
8,60,1,0.1,5000,23.226210
7,60,5,0.1,5000,23.617171
